In [1]:
#%pip install pyarrow
#%pip install pandas
#%pip install numpy
#%pip install matplotlib
#%pip install pyfinance
#%pip install yfinance
#%pip install yahoo-fin
import sys
import pyarrow.feather as feather
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pyfinance.options import BSM
import yfinance as yf
from datetime import datetime, timedelta, date
from yahoo_fin import options as op

In [11]:
def get_options_data(ticker, expiration_date):
    stock = yf.Ticker(ticker)
    options_data = stock.option_chain(expiration_date)
    return options_data

ticker_symbol = "AAPL"
expiration_date = "2023-12-22"

options_data = get_options_data(ticker_symbol, expiration_date)
print(options_data.calls)  # Access call options data
print(type(options_data.calls))
#print(options_data.puts)   # Access put options data


         contractSymbol             lastTradeDate  strike  lastPrice  bid  \
0   AAPL231222C00105000 2023-11-02 15:15:10+00:00   105.0      72.45  0.0   
1   AAPL231222C00110000 2023-11-02 14:44:01+00:00   110.0      68.20  0.0   
2   AAPL231222C00125000 2023-11-03 15:26:19+00:00   125.0      50.77  0.0   
3   AAPL231222C00135000 2023-11-13 20:02:46+00:00   135.0      50.56  0.0   
4   AAPL231222C00140000 2023-11-14 14:55:02+00:00   140.0      48.00  0.0   
5   AAPL231222C00145000 2023-11-13 20:42:15+00:00   145.0      40.98  0.0   
6   AAPL231222C00150000 2023-11-10 17:39:44+00:00   150.0      36.53  0.0   
7   AAPL231222C00155000 2023-11-15 18:09:38+00:00   155.0      34.75  0.0   
8   AAPL231222C00160000 2023-11-16 18:28:04+00:00   160.0      30.50  0.0   
9   AAPL231222C00165000 2023-11-16 17:42:53+00:00   165.0      26.20  0.0   
10  AAPL231222C00170000 2023-11-16 14:58:37+00:00   170.0      21.95  0.0   
11  AAPL231222C00175000 2023-11-16 20:22:41+00:00   175.0      16.05  0.0   

In [21]:
import pandas as pd
import yfinance as yf

def get_options_data(ticker, expiration_date):
    stock = yf.Ticker(ticker)
    options_data = stock.option_chain(expiration_date)
    return options_data

def create_options_dataframe(ticker, expiration_dates):
    # Initialize columns
    data = {'Date': [], 'Underlying Price': []}
    call_columns = []
    put_columns = []

    for expiration_date in expiration_dates:
        call_prefix = f'Call_{expiration_date}_'
        put_prefix = f'Put_{expiration_date}_'

        call_columns.extend([f'{call_prefix}{strike}' for strike in range(100, 151, 5)])
        put_columns.extend([f'{put_prefix}{strike}' for strike in range(100, 151, 5)])

    data.update({col: [] for col in call_columns + put_columns})
    print(call_columns)
    print("data on", data)

    # Add data
    for expiration_date in expiration_dates:
        options_data = get_options_data(ticker, expiration_date)

        # Extract underlying price from the first call option
        underlying_price = options_data.calls.iloc[0]['lastPrice']

        data['Date'].append(expiration_date)
        data['Underlying Price'].append(underlying_price)

        # Populate call option prices
        for strike in range(100, 151, 5):
            call_col = f'Call_{expiration_date}_{strike}'
            call_data = options_data.calls[options_data.calls['strike'] == float(strike)]['lastPrice']
            if not call_data.empty:
                data[call_col].append(float(call_data.iloc[0]))
            else:
                data[call_col].append(0)

        # Populate put option prices
        for strike in range(100, 151, 5):
            put_col = f'Put_{expiration_date}_{strike}'
            put_data = options_data.puts[options_data.puts['strike'] == float(strike)]['lastPrice']
            if not put_data.empty:
                data[put_col].append(float(put_data.iloc[0]))
            else:
                data[put_col].append(0)

    # Create DataFrame
    options_df = pd.DataFrame(data)
    return options_df

# Example usage:
ticker_symbol = "AAPL"
expiration_dates = ["2023-12-22", "2024-01-19", "2024-02-16"]

options_dataframe = create_options_dataframe(ticker_symbol, expiration_dates)
print(options_dataframe)


data on {'Date': [], 'Underlying Price': [], 'Call_2023-12-22_100': [], 'Call_2023-12-22_105': [], 'Call_2023-12-22_110': [], 'Call_2023-12-22_115': [], 'Call_2023-12-22_120': [], 'Call_2023-12-22_125': [], 'Call_2023-12-22_130': [], 'Call_2023-12-22_135': [], 'Call_2023-12-22_140': [], 'Call_2023-12-22_145': [], 'Call_2023-12-22_150': [], 'Call_2024-01-19_100': [], 'Call_2024-01-19_105': [], 'Call_2024-01-19_110': [], 'Call_2024-01-19_115': [], 'Call_2024-01-19_120': [], 'Call_2024-01-19_125': [], 'Call_2024-01-19_130': [], 'Call_2024-01-19_135': [], 'Call_2024-01-19_140': [], 'Call_2024-01-19_145': [], 'Call_2024-01-19_150': [], 'Call_2024-02-16_100': [], 'Call_2024-02-16_105': [], 'Call_2024-02-16_110': [], 'Call_2024-02-16_115': [], 'Call_2024-02-16_120': [], 'Call_2024-02-16_125': [], 'Call_2024-02-16_130': [], 'Call_2024-02-16_135': [], 'Call_2024-02-16_140': [], 'Call_2024-02-16_145': [], 'Call_2024-02-16_150': [], 'Put_2023-12-22_100': [], 'Put_2023-12-22_105': [], 'Put_2023-12

ValueError: All arrays must be of the same length